In [1]:
import numpy as np
import matplotlib.pyplot as plt
import npc_lims
from npc_sessions import DynamicRoutingSession
from dynamic_routing_analysis import decoding_utils

import os

In [ ]:
#find all DR sessions with ephys
DR_ephys_sessions = tuple(s for s in npc_lims.get_session_info() 
                          if s.is_ephys and s.is_uploaded and s.project=='DRPilotSession')

savepath=r'C:\Users\ethan.mcbride\OneDrive - Allen Institute\DR decoding results\by_session\verify_npc_analysis'
filename='verify_npc_analysis.pkl'

except_list={}

binsize=0.2

params={
    'trnum':'all',
    'n_units':[5,10,20,50,100,'all'],
    'u_min':20,
    'n_repeats':25,
    'binsize':binsize,
    'time_bins':np.arange(-0.2,0.1,binsize),
    'balance_labels':1,
    'savepath':savepath,
    'filename':filename,
}


for DR_session in DR_ephys_sessions:
    # try:
    session = DynamicRoutingSession(DR_session.id)
    print(session.id+' loaded')
    if 'structure' in session.electrodes[:].columns:
        decoding_utils.decode_context_from_units(session,params)
    else:
        print('no structure column found in electrodes table, moving to next recording')
    # except Exception as e:
    #     except_list[session.id]=e